## THC FeMoco compliation

This notebook is attempting to reproduce results from sec. IV.C of (10.1103/PRXQuantum.2.030305)[https://link.aps.org/doi/10.1103/PRXQuantum.2.030305] (here on "THC paper"), regarding surface code compliation of quantum phase estimation on the THC-encoded Li et al. FeMoco Hamiltonian.

In [2]:
from functools import partial

from qualtran.surface_code.algorithm_summary import AlgorithmSummary
from qualtran.surface_code.multi_factory import MultiFactory
from qualtran.surface_code.data_block import SimpleDataBlock
from qualtran.surface_code.ccz2t_cost_model import get_ccz2t_costs_from_grid_search, \
                                                   iter_ccz2t_factories, \
                                                   get_ccz2t_costs, \
                                                   CCZ2TFactory

# Logical data (input for grid search), from THC paper
ERR_BUDGET = 0.01
N_TOFFOLI = 6665400000 # Pag. 26
N_QUBITS = 696 # Fig. 10 
PHYS_ERR = 1e-3
N_FACTORIES = 4
CYCLE_TIME = 1

n_magic = AlgorithmSummary(toffoli_gates=N_TOFFOLI)


# Manual optimal values (output of grid search) from THC paper
DISTILLATION_L1_D = 19
DISTILLATION_L2_D = 31
DATA_D = 31
FOOTPRINT = int(4E10)
DURATION_DAYS = 3

target_factory = MultiFactory(CCZ2TFactory(DISTILLATION_L1_D, DISTILLATION_L2_D), N_FACTORIES)
target_data_block = SimpleDataBlock(DATA_D)

## Using manual values for factory and data block

In [5]:
distillation_error = target_factory.distillation_error(n_magic, PHYS_ERR)
data_error = target_data_block.data_error(n_algo_qubits=N_QUBITS, 
                                          n_cycles=target_factory.n_cycles(n_magic),
                                          phys_err=PHYS_ERR)

print(f'distillation error: {distillation_error:.3%}') # ref: 0.1% per 1e10 Toffolis
print(f'data error: {data_error:.3%}')

manual_cost = get_ccz2t_costs(
    n_magic=n_magic,
    n_algo_qubits=N_QUBITS,
    phys_err=PHYS_ERR,
    cycle_time_us=CYCLE_TIME,
    factory=target_factory,
    data_block=target_data_block
)

print(f'failure probability: {manual_cost.failure_prob:.3%}')
print(f'wall time: {manual_cost.duration_hr/24} days') # ref: 3 days
print(f'footprint: {manual_cost.footprint*1e-6:.2f} million qubits') # ref: 4 million qubits 

distillation error: 0.069%
data error: 0.373%
failure probability: 0.442%
wall time: 4.1369453125 days
footprint: 2.81 million qubits


## Grid search

In [7]:
best_cost, best_factory, best_data_block = get_ccz2t_costs_from_grid_search(
    n_magic=n_magic,
    n_algo_qubits=N_QUBITS,
    error_budget=ERR_BUDGET,
    phys_err=PHYS_ERR,
    factory_iter=partial(iter_ccz2t_factories, n_factories=N_FACTORIES),
    cost_function=(lambda pc: pc.duration_hr)
)

print(best_cost)
print(best_factory)
print(best_data_block)
print()

distillation_error = best_factory.distillation_error(n_magic, PHYS_ERR)
data_error = best_data_block.data_error(n_algo_qubits=N_QUBITS, 
                                        n_cycles=best_factory.n_cycles(n_magic),
                                        phys_err=PHYS_ERR)

print(f'distillation error: {distillation_error:.3%}') # ref: 0.1% per 1e10 Toffolis
print(f'data error: {data_error:.3%}')
print(f'failure probability: {best_cost.failure_prob:.3%}')
print(f'wall time: {best_cost.duration_hr/24} days') # ref: 3 days
print(f'footprint: {best_cost.footprint*1e-6:.2f} million qubits') # ref: 4 million qubits 

PhysicalCost(failure_prob=0.007725395132201774, footprint=2933032, duration_hr=89.1034375)
MultiFactory(base_factory=CCZ2TFactory(distillation_l1_d=17, distillation_l2_d=29), n_factories=4)
SimpleDataBlock(data_d=33, routing_overhead=0.5)

distillation error: 0.739%
data error: 0.033%
failure probability: 0.773%
wall time: 3.7126432291666664 days
footprint: 2.93 million qubits


The optimization result does not match the THC paper results exactly. 
The cost model used is similar, but the results from the THC paper include a detailed study of the lattice surgery layout. 
Here, instead, we just assume a 50% space overhead for routing and we compute the time as only limited by the CCZ generation rate.
The shorter runtime, at a cost of an increased number of physical qubits, might be due to this.